Some of the code in this lab was taken from the solution at https://github.com/learn-co-curriculum/dsc-tuning-neural-networks-with-normalization-lab/tree/solution

# Tuning Neural Networks with Normalization - Lab

## Introduction

For this lab on initialization and optimization, you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives
You will be able to:
* Build a neural network using Keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Loading the data

The data we'll be working with is data related to Facebook posts published during the year of 2014 on the Facebook page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set, `dataset_Facebook.csv`, and delete any rows with missing data. Afterwards, briefly preview the data.

In [2]:
# this code was taken from the solution at https://github.com/learn-co-curriculum/dsc-tuning-neural-networks-with-normalization-lab/tree/solution
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', sep=';', header=0)
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(500, 19)
(495, 19)


,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393



## Defining the Problem

Define X and Y and perform a train-validation-test split.

X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category

Y will be the `like` column.

In [3]:
df.columns

Index(['Page total likes', 'Type', 'Category', 'Post Month', 'Post Weekday',
       'Post Hour', 'Paid', 'Lifetime Post Total Reach',
       'Lifetime Post Total Impressions', 'Lifetime Engaged Users',
       'Lifetime Post Consumers', 'Lifetime Post Consumptions',
       'Lifetime Post Impressions by people who have liked your Page',
       'Lifetime Post reach by people who like your Page',
       'Lifetime People who have liked your Page and engaged with your post',
       'comment', 'like', 'share', 'Total Interactions'],
      dtype='object')

In [4]:
#Your code here; define the problem.
X = df[['Page total likes', 'Post Month', 'Post Weekday', 'Post Hour', 'Paid']]
dummies1 = pd.get_dummies(df['Type'], prefix='type', drop_first=True)
dummies2 = pd.get_dummies(df['Category'], prefix='cat', drop_first=True)
X = pd.concat([X, dummies1, dummies2], axis=1)

Y = df['like']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=123)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=123)

## Building a Baseline Model

Next, build a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model.

In [5]:
#Simply run this code block, later you'll modify this model to tune the performance
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

### Evaluating the Baseline

Evaluate the baseline model for the training and validation sets.

In [6]:
# this code was taken from the solution at https://github.com/learn-co-curriculum/dsc-tuning-neural-networks-with-normalization-lab/tree/solution
#Your code here; evaluate the model with MSE
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train - Y_train)**2)
MSE_val = np.mean((pred_val - Y_val)**2)

print('MSE_train:', MSE_train)
print('MSE_val:', MSE_val)

MSE_train: nan
MSE_val: nan


In [7]:
#Your code here; inspect the loss function through the history object
print(hist.history['loss'][-1])
hist.history['val_loss'][-1]

nan


nan

> Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.

## Normalize the Input Data

Normalize the input features by subtracting each feature mean and dividing by the standard deviation in order to transform each into a standard normal distribution. Then recreate the train-validate-test sets with the transformed input data.

In [8]:
## standardize/categorize
for col in X.columns:
    X[col] = (X[col] - X[col].mean()) / X[col].std()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=123)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=123)

## Refit the Model and Reevaluate

Great! Now refit the model and once again assess it's performance on the training and validation sets.

In [9]:
#Your code here; refit a model as shown above
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [10]:
#Rexamine the loss function
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train - Y_train)**2)
MSE_val = np.mean((pred_val - Y_val)**2)

print('MSE_train:', MSE_train)
print('MSE_val:', MSE_val)

print(hist.history['loss'][-1])
hist.history['val_loss'][-1]

MSE_train: nan
MSE_val: nan
nan


nan

> Note that you still haven't achieved convergence! From here, it's time to normalize the output data.

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [11]:
#Your code here: redefine Y after normalizing the data.
Y = (Y - Y.mean())/Y.std()

In [12]:
#Your code here; create training and validation sets as before. Use random seed 123.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=123)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=123)

In [13]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. (See our code snippet above!)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

Again, reevaluate the updated model.

In [14]:

pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train - Y_train)**2)
MSE_val = np.mean((pred_val - Y_val)**2)

print('MSE_train:', MSE_train)
print('MSE_val:', MSE_val)



MSE_train: 1.1434353354328057
MSE_val: 0.46979866513327917


In [15]:
print(hist.history['loss'][-1])
hist.history['val_loss'][-1]

1.1495444774627686


0.46979867626902877

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers

Below, take a look at the code provided to see how to modify the neural network to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe is the strongest.

##  He Initialization

In [16]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=0)

In [17]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [18]:
print(MSE_train)
print(MSE_val)

1.1832366228984734
0.4699831216764756


In [19]:
mse_dict = {'He':[MSE_train, MSE_val]}


## Lecun Initialization

In [20]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [21]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [22]:
print(MSE_train)
print(MSE_val)

1.1395004980296721
0.43188763857163714


In [23]:
mse_dict['Lecun'] = [MSE_train, MSE_val]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop

In [24]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [25]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [26]:
print(MSE_train)
print(MSE_val)

1.145108657337657
0.45986380205013144


In [27]:
mse_dict['RMSprop'] = [MSE_train, MSE_val]

## Adam

In [28]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [29]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [30]:
print(MSE_train)
print(MSE_val)

1.1362346303008257
0.4279757023051183


In [31]:
mse_dict['Adam'] = [MSE_train, MSE_val]

## Learning Rate Decay with Momentum


In [32]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [33]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [34]:
print(MSE_train)
print(MSE_val)

1.0093602843056793
0.694877369250468


In [35]:
mse_dict['LRDwM'] = [MSE_train, MSE_val]

## Selecting a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [47]:
#Your code here
sorted(mse_dict.items(), key = lambda x: x[1][1])

[('Adam', [1.1362346303008257, 0.4279757023051183]),
 ('Lecun', [1.1395004980296721, 0.43188763857163714]),
 ('RMSprop', [1.145108657337657, 0.45986380205013144]),
 ('He', [1.1832366228984734, 0.4699831216764756]),
 ('LRDwM', [1.0093602843056793, 0.694877369250468])]

In [48]:
# Adam Model
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [55]:
Y_hat = model.predict(X_test).reshape(-1)
mse = np.mean((Y_hat - Y_test)**2)
mse

0.8115848247607781

## Summary  

In this lab, you worked to ensure your model converged properly. Additionally, you also investigated the impact of varying initialization and optimization routines.